In [1]:
import pandas as pd
import os
import sys

In [2]:
file_path = 'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'

In [3]:
dim_cols_dict= {
                'item':{'key':['item_id'],
						'incols':['item_id','dept_id','cat_id'],
						'outcols':['item_id','dept_id','cat_id']
						},
               'store':{'key':['store_id'],
			            'incols':['store_id','state_id'],
						'outcols': ['store_id','state_id'] 
						},
              'datedim':{'key':['date_sk']
                        }
              }

In [4]:
fact_model = { 'f_sales_trans':{'dims':['item','date','store'],
                                'Msrs' : ['qty','price'],
                                'DegDim' :['trans_id']
                               }
             }

In [5]:
read_fact_input = pd.read_csv(file_path + 'sales_train_validation.csv')

In [6]:
read_fact_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [13]:
def read_dim_for_fact(dim_cols_dict,dim_type):
  dim_name = 'dim_' + dim_type
  dim_key_cols = dim_cols_dict[dim_type]['key']
  dim_in_all_cols = dim_cols_dict[dim_type]['incols']
  dim_out_cols = dim_cols_dict[dim_type]['outcols']  
  file_path = 'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'
  #file_name = 'sales_train_validation.csv'
  dim_sk = dim_type + '_sk'
  dim_max_sk_csv = dim_name + '_max_sk.csv'
  dim_idx_lkp_csv = dim_name + '_idx_lkp.csv'
  dim_csv = dim_name + '.csv'
  dim_hash = dim_type + '_hash_key'
  dim_out_all_cols = list(dim_out_cols)
  dim_idx_cols = list(dim_key_cols)
  dim_out_all_cols.extend([dim_sk,dim_hash,'rec_hash'])
  dim_idx_cols.append(dim_sk)
  df_dim_idx_lkp = pd.read_csv(file_path + dim_idx_lkp_csv,index_col=dim_hash)
  return(df_dim_idx_lkp)

In [9]:
def add_sk_to_fact(df_fact,dim_cols_dict,dim_type):
  dim_hash = dim_type + '_hash_key'
  dim_key_cols = dim_cols_dict[dim_type]['key']
  dim_in_all_cols = dim_cols_dict[dim_type]['incols']
  dim_in_all_cols.append(dim_hash)
  dim_sk = dim_type + '_sk'
  if dim_sk not in df_fact.columns:
    df_fact[dim_hash] = pd.util.hash_pandas_object(df_fact[dim_key_cols],index=False)
    df_dim_idx_lkp = read_dim_for_fact(dim_cols_dict,dim_type)
    def lookup_fun(x):
      try:
        return(df_dim_idx_lkp.at[x,dim_sk])
      except:
        return(-1)
    df_fact.insert(1,dim_sk,df_fact[dim_hash].apply(lambda x:lookup_fun(x)))
    #df_fact[dim_sk] = df_fact[dim_hash].apply(lambda x:lookup_fun(x))
    df_dim_placeholder = df_fact[dim_key_cols][df_fact[dim_sk] == -1]
    df_fact.drop(columns=dim_in_all_cols,inplace=True)
    return(df_fact,df_dim_placeholder)
  else:
    return(df_fact,pd.DataFrame())

In [14]:
read_fact_input,df_store_placeholder = add_sk_to_fact(read_fact_input,dim_cols_dict,'store')

In [15]:
read_fact_input,df_item_placeholder = add_sk_to_fact(read_fact_input,dim_cols_dict,'item')

In [16]:
read_fact_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1916 entries, id to d_1913
dtypes: int64(1915), object(1)
memory usage: 445.7+ MB


In [19]:
read_fact_input.item_sk.unique()

array([   1,    2,    3, ..., 3047, 3048, 3049], dtype=int64)

In [20]:
read_fact_input.to_csv(file_path + 'f_sales_trans_sk_mapped_test.csv',index=False)